In [1]:
 sc.applicationId

'local-1638884076952'

In [2]:
from pyspark.sql.functions import isnan,isnull,when,count,col,countDistinct,explode,array,struct,lit,sum
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from typing import Iterable 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# data = spark.read.format("csv").option("header",True).load("/home/huan/Downloads/train.csv")
data = sqlContext.read.format('com.databricks.spark.csv')\
       .option('header', 'true')\
       .option('delimiter', ',').load("/home/huan/Downloads/train.csv")

In [4]:
data = data.select([F.col(col).alias(col.replace(' ', '_')) for col in data.columns])

In [5]:
#So dong du lieuj
data.count()

103904

In [6]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

21/12/07 05:34:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, Gender, Customer Type, Age, Type of Travel, Class, Flight Distance, Inflight wifi service, Departure/Arrival time convenient, Ease of Online booking, Gate location, Food and drink, Online boarding, Seat comfort, Inflight entertainment, On-board service, Leg room service, Baggage handling, Checkin service, Inflight service, Cleanliness, Departure Delay in Minutes, Arrival Delay in Minutes, satisfaction
 Schema: _c0, id, Gender, Customer Type, Age, Type of Travel, Class, Flight Distance, Inflight wifi service, Departure/Arrival time convenient, Ease of Online booking, Gate location, Food and drink, Online boarding, Seat comfort, Inflight entertainment, On-board service, Leg room service, Baggage handling, Checkin service, Inflight service, Cleanliness, Departure Delay in Minutes, Arrival Delay in Minutes, satisfaction
Expected: _c0 but found: 
CSV file: file:///home/huan/Downloads/train.csv

+---+---+------+-------------+---+--------------+-----+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+------------+
|_c0| id|Gender|Customer_Type|Age|Type_of_Travel|Class|Flight_Distance|Inflight_wifi_service|Departure/Arrival_time_convenient|Ease_of_Online_booking|Gate_location|Food_and_drink|Online_boarding|Seat_comfort|Inflight_entertainment|On-board_service|Leg_room_service|Baggage_handling|Checkin_service|Inflight_service|Cleanliness|Departure_Delay_in_Minutes|Arrival_Delay_in_Minutes|satisfaction|
+---+---+------+-------------+---+--------------+-----+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+------

In [7]:
data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).toPandas()

21/12/07 05:34:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, Gender, Customer Type, Age, Type of Travel, Class, Flight Distance, Inflight wifi service, Departure/Arrival time convenient, Ease of Online booking, Gate location, Food and drink, Online boarding, Seat comfort, Inflight entertainment, On-board service, Leg room service, Baggage handling, Checkin service, Inflight service, Cleanliness, Departure Delay in Minutes, Arrival Delay in Minutes, satisfaction
 Schema: _c0, id, Gender, Customer Type, Age, Type of Travel, Class, Flight Distance, Inflight wifi service, Departure/Arrival time convenient, Ease of Online booking, Gate location, Food and drink, Online boarding, Seat comfort, Inflight entertainment, On-board service, Leg room service, Baggage handling, Checkin service, Inflight service, Cleanliness, Departure Delay in Minutes, Arrival Delay in Minutes, satisfaction
Expected: _c0 but found: 
CSV file: file:///home/huan/Downloads/train.csv

,_c0,id,Gender,Customer_Type,Age,Type_of_Travel,Class,Flight_Distance,Inflight_wifi_service,Departure/Arrival_time_convenient,...,Inflight_entertainment,On-board_service,Leg_room_service,Baggage_handling,Checkin_service,Inflight_service,Cleanliness,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes,satisfaction
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,310,0


In [8]:
data = data.drop(*["_c0","id"])
# data.na.drop()
data = data.fillna({"Arrival_Delay_in_Minutes":0})

In [9]:
data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).toPandas()

,Gender,Customer_Type,Age,Type_of_Travel,Class,Flight_Distance,Inflight_wifi_service,Departure/Arrival_time_convenient,Ease_of_Online_booking,Gate_location,...,Inflight_entertainment,On-board_service,Leg_room_service,Baggage_handling,Checkin_service,Inflight_service,Cleanliness,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes,satisfaction
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#Cau truc du lieu:
data.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Customer_Type: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Type_of_Travel: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Flight_Distance: string (nullable = true)
 |-- Inflight_wifi_service: string (nullable = true)
 |-- Departure/Arrival_time_convenient: string (nullable = true)
 |-- Ease_of_Online_booking: string (nullable = true)
 |-- Gate_location: string (nullable = true)
 |-- Food_and_drink: string (nullable = true)
 |-- Online_boarding: string (nullable = true)
 |-- Seat_comfort: string (nullable = true)
 |-- Inflight_entertainment: string (nullable = true)
 |-- On-board_service: string (nullable = true)
 |-- Leg_room_service: string (nullable = true)
 |-- Baggage_handling: string (nullable = true)
 |-- Checkin_service: string (nullable = true)
 |-- Inflight_service: string (nullable = true)
 |-- Cleanliness: string (nullable = true)
 |-- Departure_Delay_in_Minutes: string (nullable = true)

In [11]:
#convert datatype to int
data = data.withColumn('Age',data['Age'].cast('int'))
data = data.withColumn('Flight_Distance',data['Flight_Distance'].cast('int'))
data = data.withColumn('Inflight_wifi_service',data['Inflight_wifi_service'].cast('int'))
data = data.withColumn('Departure/Arrival_time_convenient',data['Departure/Arrival_time_convenient'].cast('int'))
data = data.withColumn('Ease_of_Online_booking',data['Ease_of_Online_booking'].cast('int'))
data = data.withColumn('Gate_location',data['Gate_location'].cast('int'))
data = data.withColumn('Food_and_drink',data['Food_and_drink'].cast('int'))
data = data.withColumn('Online_boarding',data['Online_boarding'].cast('int'))
data = data.withColumn('Seat_comfort',data['Seat_comfort'].cast('int'))
data = data.withColumn('On-board_service',data['On-board_service'].cast('int'))
data = data.withColumn('Leg_room_service',data['Leg_room_service'].cast('int'))
data = data.withColumn('Baggage_handling',data['Baggage_handling'].cast('int'))
data = data.withColumn('Checkin_service',data['Checkin_service'].cast('int'))
data = data.withColumn('Inflight_service',data['Inflight_service'].cast('int'))
data = data.withColumn('Cleanliness',data['Cleanliness'].cast('int'))
data = data.withColumn('Departure_Delay_in_Minutes',data['Departure_Delay_in_Minutes'].cast('int'))
data = data.withColumn('Inflight_entertainment',data['Inflight_entertainment'].cast('int'))
data = data.withColumn('Arrival_Delay_in_Minutes',data['Arrival_Delay_in_Minutes'].cast('int'))


In [12]:
data.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Customer_Type: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Type_of_Travel: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Flight_Distance: integer (nullable = true)
 |-- Inflight_wifi_service: integer (nullable = true)
 |-- Departure/Arrival_time_convenient: integer (nullable = true)
 |-- Ease_of_Online_booking: integer (nullable = true)
 |-- Gate_location: integer (nullable = true)
 |-- Food_and_drink: integer (nullable = true)
 |-- Online_boarding: integer (nullable = true)
 |-- Seat_comfort: integer (nullable = true)
 |-- Inflight_entertainment: integer (nullable = true)
 |-- On-board_service: integer (nullable = true)
 |-- Leg_room_service: integer (nullable = true)
 |-- Baggage_handling: integer (nullable = true)
 |-- Checkin_service: integer (nullable = true)
 |-- Inflight_service: integer (nullable = true)
 |-- Cleanliness: integer (nullable = true)
 |-- Departure_Delay_in_Minutes: integer 

In [13]:
data.take(2)

[Row(Gender='Male', Customer_Type='Loyal Customer', Age=13, Type_of_Travel='Personal Travel', Class='Eco Plus', Flight_Distance=460, Inflight_wifi_service=3, Departure/Arrival_time_convenient=4, Ease_of_Online_booking=3, Gate_location=1, Food_and_drink=5, Online_boarding=3, Seat_comfort=5, Inflight_entertainment=5, On-board_service=4, Leg_room_service=3, Baggage_handling=4, Checkin_service=4, Inflight_service=5, Cleanliness=5, Departure_Delay_in_Minutes=25, Arrival_Delay_in_Minutes=18, satisfaction='neutral or dissatisfied'),
 Row(Gender='Male', Customer_Type='disloyal Customer', Age=25, Type_of_Travel='Business travel', Class='Business', Flight_Distance=235, Inflight_wifi_service=3, Departure/Arrival_time_convenient=2, Ease_of_Online_booking=3, Gate_location=3, Food_and_drink=1, Online_boarding=3, Seat_comfort=1, Inflight_entertainment=1, On-board_service=1, Leg_room_service=5, Baggage_handling=3, Checkin_service=1, Inflight_service=4, Cleanliness=1, Departure_Delay_in_Minutes=1, Arri

In [14]:
#summary statistic for numeric variables
# data.describe().show()
data.toPandas().describe()

,Age,Flight_Distance,Inflight_wifi_service,Departure/Arrival_time_convenient,Ease_of_Online_booking,Gate_location,Food_and_drink,Online_boarding,Seat_comfort,Inflight_entertainment,On-board_service,Leg_room_service,Baggage_handling,Checkin_service,Inflight_service,Cleanliness,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes
count,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000
mean,39.379706,1189.448375,2.729683,3.060296,2.756901,2.976883,3.202129,3.250375,3.439396,3.358158,3.382363,3.351055,3.631833,3.304290,3.640428,3.286351,14.815618,15.133392
std,15.114964,997.147281,1.327829,1.525075,1.398929,1.277621,1.329533,1.349509,1.319088,1.332991,1.288354,1.315605,1.180903,1.265396,1.175663,1.312273,38.230901,38.649776
min,7.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,27.000000,414.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000
50%,40.000000,843.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.000000,0.000000
75%,51.000000,1743.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,12.000000,13.000000
max,85.000000,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1592.000000,1584.000000


In [15]:
# string_columns =[item[0] for item in data.dtypes if item[1].startswith('string')]
# for col in data.columns:
#     if not col in string_columns:
#         medi = np.median(data.select(col).rdd.flatMap(lambda x:x).collect())
#         print("Median of ", col,":", medi)

In [16]:
# string_columns =[item[0] for item in data.dtypes if item[1].startswith('string')]
# for col in data.columns:
#     if not col in string_columns:
#         medi = np.min(data.select(col).rdd.flatMap(lambda x:x).collect())
#         print("min of ", col,":", medi)

In [17]:
# string_columns =[item[0] for item in data.dtypes if item[1].startswith('string')]
# for col in data.columns:
#     if not col in string_columns:
#         medi = np.mean(data.select(col).rdd.flatMap(lambda x:x).collect())
#         print("Mean of ", col,":", medi)

In [18]:
# string_columns =[item[0] for item in data.dtypes if item[1].startswith('string')]
# for col in data.columns:
#     if not col in string_columns:
#         medi = np.max(data.select(col).rdd.flatMap(lambda x:x).collect())
#         print("Max of ", col,":", medi)

In [19]:
# import seaborn as sns
# sns.set(style="ticks")
# data= sns.load_dataset("iris")
# sns.pairplot(data, hue="species")
# plt.show()

In [20]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# sum_age = data.groupBy('Age').count().sort("Age").collect()
# age_count = list(map(lambda x: int(x[0]), sum_age))
# class_count = list(map(lambda x: x[1],sum_age))

# index = np.arange(len(age_count))

# fig, ax = plt.subplots(figsize=(15,6))
# ax.set_xlabel('Age')
# ax.set_ylabel('Count')
# ax.bar(age_count, class_count, align='center', width=0.5)
# for i,v in enumerate(class_count):
#     plt.text(x=i+7, y = v  ,s=f"{v}", fontdict=dict(fontsize=10),rotation='vertical', ha='center' )
# plt.show()

In [21]:
# def ChartCountPlot(data,x_axis,hue):
#     plt.figure(figsize=(20,10))
#     sns.countplot(x_axis,data=data,hue=hue)
#     plt.title("Chart of "+x_axis +" by "+hue)
#     plt.show()

In [22]:
# dt_pd = data.toPandas()

In [23]:
# ChartCountPlot(dt_pd,'Age','satisfaction')

In [24]:
# gender_count = data.groupBy('Gender').count().collect()
# gender_ = list(map(lambda x: x[0],gender_count))
# count_ = list(map(lambda x: int(x[1]),gender_count))

# fig, ax = plt.subplots(figsize=(10,5))
# ind = np.arange(len(count_))
# for i, v in enumerate(count_):
#     ax.text(v + 3, i, str(v), fontweight='bold', color='C0')
#     ax.barh(ind, count_, color = 'C0')
#     ax.set_yticks(ind)
#     ax.set_yticklabels(gender_)
#     plt.show

In [25]:
# satisfaction = data.groupBy('Gender', 'satisfaction').count().sort('Gender').collect()
# gender_ = list(map(lambda x: x[0],satisfaction))
# satisfaction_ = list(map(lambda x: x[1],satisfaction))
# count_ = list(map(lambda x: int(x[2]),satisfaction))


# y_pos = np.arange(len(gender_))
# error = np.random.rand(len(gender_))
# clr = ('C0', 'C1')
# fig, ax = plt.subplots(figsize=(10,5))
# h = ax.barh(y_pos, count_, xerr=error, align='center', 
# alpha=0.4, label=gender_, color=clr)
# plt.yticks(y_pos, gender_)
# ax.set_title('Gender vs satisfaction')
# plt.xlabel('Count')
# plt.ylabel('Gender')
# plt.legend(h,set(satisfaction_), loc=(0,1)
#           )
# for i,v in enumerate(count_):
#     ax.text(v + 3, i, str(v), fontweight='bold', color='C0')
# plt.show()

In [26]:
# satisfaction = data.groupBy('Class', 'satisfaction').count().sort('Class').collect()
# class_ = list(map(lambda x: x[0],satisfaction))
# satisfaction_ = list(map(lambda x: x[1],satisfaction))
# count_ = list(map(lambda x: int(x[2]),satisfaction))


# y_pos = np.arange(len(class_))
# error = np.random.rand(len(class_))
# clr = ('C0', 'C1')
# fig, ax = plt.subplots(figsize=(13,5))
# h = ax.barh(y_pos, count_, xerr=error, align='center', 
# alpha=0.4, label=class_, color=clr)
# plt.yticks(y_pos, class_)
# ax.set_title('Class vs satisfaction')
# plt.xlabel('Count')
# plt.ylabel('Class')
# plt.legend(h,sorted(set(satisfaction_)))
# for i,v in enumerate(count_):
#     ax.text(v + 3, i, str(v), fontweight='bold', color='C0')
# plt.show()

In [27]:
# satisfaction = data.groupBy('Customer_Type', 'satisfaction').count().sort('Customer_Type').collect()
# gender_ = list(map(lambda x: x[0],satisfaction))
# satisfaction_ = list(map(lambda x: x[1],satisfaction))
# count_ = list(map(lambda x: int(x[2]),satisfaction))


# y_pos = np.arange(len(gender_))
# error = np.random.rand(len(gender_))
# clr = ('C0', 'C1')
# fig, ax = plt.subplots(figsize=(13,5))
# h = ax.barh(y_pos, count_, xerr=error, align='center', 
# alpha=0.4, label=gender_, color=clr)
# plt.yticks(y_pos, gender_)
# ax.set_title('Customer Type vs satisfaction')
# plt.xlabel('Count')
# plt.ylabel('Customer Type')
# plt.legend(h,set(satisfaction_), loc=(0,1)
#           )
# for i,v in enumerate(count_):
#     ax.text(v + 3, i, str(v), fontweight='bold', color='C0')
# plt.show()

In [28]:
# satisfaction = data.groupBy('Type_of_Travel', 'satisfaction').count().sort('Type_of_Travel','count').collect()
# type_ = list(map(lambda x: x[0],satisfaction))
# satisfaction_ = list(map(lambda x: x[1],satisfaction))
# count_ = list(map(lambda x: int(x[2]),satisfaction))


# y_pos = np.arange(len(type_))
# error = np.random.rand(len(type_))
# clr = ('C0', 'C1')
# fig, ax = plt.subplots(figsize=(18,5))
# h = ax.barh(y_pos, count_, xerr=error, align='center', 
# alpha=0.4, label=type_, color=clr)
# plt.yticks(y_pos, type_)
# ax.set_title('Type Travel vs satisfaction')
# plt.xlabel('Count')
# plt.ylabel('Type of Travel')

# for i,(v,k) in enumerate(zip(count_,satisfaction_)):
#     ax.text(v + 3, i, str(v)+' with '+str(k), fontweight='bold', color='C0')
# plt.show()

In [29]:
# G=sns.FacetGrid(data=data.toPandas(), col='satisfaction',height=5)
# G.map(sns.distplot, 'Flight_Distance')
# plt.show()


In [30]:
# G=sns.FacetGrid(data=data.toPandas(), col='satisfaction',height=5)
# G.map(sns.distplot, 'Arrival_Delay_in_Minutes')
# plt.show()

In [31]:
# data

In [32]:
# plt.figure(figsize=(12,12))
# sns.heatmap(dt_pd.corr(method='pearson'), annot=True)

In [33]:
data = data.drop("Departure_Delay_in_Minutes")
data = data.drop("Ease_of_Online_booking")

In [34]:
# def Balance(data):
#     Range = int(len(data[data['satisfaction']=='neutral or dissatisfied'])) - int(len(data[data['satisfaction']=='satisfied']))
#     if Range >0:
#         index = data[data['satisfaction'] == 'neutral or dissatisfied'].index.values
#         index = index.tolist()
#         array = random.sample(index, Range)
#         data = data.drop(labels=array, axis=0)
#     else:
#             index = data[data['satisfaction'] == 'satisfied'].index.values
#             index = index.tolist()
#             array = random.sample(index, -Range)
#             data = data.drop(labels=array, axis=0)
#     return data

In [35]:
# row_max = data.agg({'Age': 'max'},).collect()[0]
# maxx = row_max['max(Age)']
# row_min = data.agg({'Age': 'min'},).collect()[0]
# minn = row_min['min(Age)']
# #so gio = 10
# dorong=(maxx-minn)/10
# print(maxx,minn,dorong)

In [36]:
data = data.withColumn('Age', when(data.Age <15 ,"[7..15)") \
                           .when(data.Age <23,"[15..23)") \
                           .when(data.Age <31,"[23..31)") \
                           .when(data.Age <39,"[31..39)") \
                           .when(data.Age <47,"[39..47)") \
                           .when(data.Age <55,"[47..55)") \
                           .when(data.Age <63,"[55..63)") \
                           .when(data.Age <71,"[63..71)") \
                           .when(data.Age <79,"[71..79)") \
                           .otherwise("[79..)"))
# data.select(data['Age']).collect()

In [37]:
# data.groupBy('Age').count().show()

In [38]:
# row_max = data.agg({'Flight_Distance': 'max'},).collect()[0]
# maxx = row_max['max(Flight_Distance)']
# row_min = data.agg({'Flight_Distance': 'min'},).collect()[0]
# minn = row_min['min(Flight_Distance)']
# #so gio = 10
# dorong=(maxx-minn)/10
# print("do rong:",dorong)

In [39]:
data = data.withColumn('Flight_Distance', when(data['Flight_Distance'] <526 ,"[31...526)") \
                           .when(data['Flight_Distance'] <  1021,"[526...1021)") \
                           .when(data['Flight_Distance'] <  1516,"[1021...1516)") \
                           .when(data['Flight_Distance'] <  2011,"[1516...2011)") \
                           .when(data['Flight_Distance'] <  2506,"[2011...2506)") \
                           .when(data['Flight_Distance'] <  3001 ,"[2506...3001)") \
                           .when(data['Flight_Distance'] <  3496,"[3001...3496)") \
                           .when(data['Flight_Distance'] <  3991,"[3496...3991)") \
                           .when(data['Flight_Distance'] <  4486,"[3991...4486)") \
                           .otherwise("[4486...)"))
# data.select(data['Flight_Distance']).collect()

In [40]:
# data.groupBy('Flight_Distance').count().show()

In [41]:
# row_max = data.agg({'Arrival_Delay_in_Minutes': 'max'},).collect()[0]
# maxx = row_max['max(Arrival_Delay_in_Minutes)']
# row_min = data.agg({'Arrival_Delay_in_Minutes': 'min'},).collect()[0]
# minn = row_min['min(Arrival_Delay_in_Minutes)']
# dorong=(maxx-minn)/10
# print(maxx, minn ,dorong)

In [42]:
data = data.withColumn('Arrival_Delay_in_Minutes', when(data['Arrival_Delay_in_Minutes'] <1 ,"[..0)") \
                           .when(data['Arrival_Delay_in_Minutes'] <  1021,"[1..159)") \
                           .when(data['Arrival_Delay_in_Minutes'] <  1516,"[159...317)") \
                           .when(data['Arrival_Delay_in_Minutes'] <  2011,"[317...475)") \
                           .when(data['Arrival_Delay_in_Minutes'] <  2506,"[475...633)") \
                           .when(data['Arrival_Delay_in_Minutes'] <  3001,"[633...791)") \
                           .when(data['Arrival_Delay_in_Minutes'] <  3496,"[791...949)") \
                           .when(data['Arrival_Delay_in_Minutes'] <  3991,"[949...1107)") \
                           .when(data['Arrival_Delay_in_Minutes'] <  4486,"[1107...1265)") \
                           .otherwise("[1265...)"))

In [43]:
data.groupBy('Arrival_Delay_in_Minutes').count().show()

+------------------------+-----+
|Arrival_Delay_in_Minutes|count|
+------------------------+-----+
|                   [..0)|58469|
|             [317...475)|    1|
|                [1..159)|45433|
|             [159...317)|    1|
+------------------------+-----+



In [44]:
#convert datatype to string
data = data.withColumn('Age',data['Age'].cast('string'))
data = data.withColumn('Flight_Distance',data['Flight_Distance'].cast('string'))
data = data.withColumn('Inflight_wifi_service',data['Inflight_wifi_service'].cast('string'))
data = data.withColumn('Departure/Arrival_time_convenient',data['Departure/Arrival_time_convenient'].cast('string'))
data = data.withColumn('Gate_location',data['Gate_location'].cast('string'))
data = data.withColumn('Food_and_drink',data['Food_and_drink'].cast('string'))
data = data.withColumn('Online_boarding',data['Online_boarding'].cast('string'))
data = data.withColumn('Seat_comfort',data['Seat_comfort'].cast('string'))
data = data.withColumn('On-board_service',data['On-board_service'].cast('string'))
data = data.withColumn('Leg_room_service',data['Leg_room_service'].cast('string'))
data = data.withColumn('Baggage_handling',data['Baggage_handling'].cast('string'))
data = data.withColumn('Checkin_service',data['Checkin_service'].cast('string'))
data = data.withColumn('Inflight_service',data['Inflight_service'].cast('string'))
data = data.withColumn('Cleanliness',data['Cleanliness'].cast('string'))
data = data.withColumn('Inflight_entertainment',data['Inflight_entertainment'].cast('string'))
data = data.withColumn('Arrival_Delay_in_Minutes',data['Arrival_Delay_in_Minutes'].cast('string'))


In [45]:
data.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Customer_Type: string (nullable = true)
 |-- Age: string (nullable = false)
 |-- Type_of_Travel: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Flight_Distance: string (nullable = false)
 |-- Inflight_wifi_service: string (nullable = true)
 |-- Departure/Arrival_time_convenient: string (nullable = true)
 |-- Gate_location: string (nullable = true)
 |-- Food_and_drink: string (nullable = true)
 |-- Online_boarding: string (nullable = true)
 |-- Seat_comfort: string (nullable = true)
 |-- Inflight_entertainment: string (nullable = true)
 |-- On-board_service: string (nullable = true)
 |-- Leg_room_service: string (nullable = true)
 |-- Baggage_handling: string (nullable = true)
 |-- Checkin_service: string (nullable = true)
 |-- Inflight_service: string (nullable = true)
 |-- Cleanliness: string (nullable = true)
 |-- Arrival_Delay_in_Minutes: string (nullable = false)
 |-- satisfaction: string (nullable = true)



In [46]:
data.select('satisfaction').distinct().collect()

[Row(satisfaction='neutral or dissatisfied'), Row(satisfaction='satisfied')]

In [47]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
t={"neutral or dissatisfied":0,"satisfied":1}
udf_f= udf(lambda x:t[x], StringType())
data=data.withColumn("y", udf_f("satisfaction")).drop("satisfaction")

In [48]:
data.select('y').distinct().collect()

[Row(y='0'), Row(y='1')]

In [49]:
data.take(2)

[Row(Gender='Male', Customer_Type='Loyal Customer', Age='[7..15)', Type_of_Travel='Personal Travel', Class='Eco Plus', Flight_Distance='[31...526)', Inflight_wifi_service='3', Departure/Arrival_time_convenient='4', Gate_location='1', Food_and_drink='5', Online_boarding='3', Seat_comfort='5', Inflight_entertainment='5', On-board_service='4', Leg_room_service='3', Baggage_handling='4', Checkin_service='4', Inflight_service='5', Cleanliness='5', Arrival_Delay_in_Minutes='[1..159)', y='0'),
 Row(Gender='Male', Customer_Type='disloyal Customer', Age='[23..31)', Type_of_Travel='Business travel', Class='Business', Flight_Distance='[31...526)', Inflight_wifi_service='3', Departure/Arrival_time_convenient='2', Gate_location='3', Food_and_drink='1', Online_boarding='3', Seat_comfort='1', Inflight_entertainment='1', On-board_service='1', Leg_room_service='5', Baggage_handling='3', Checkin_service='1', Inflight_service='4', Cleanliness='1', Arrival_Delay_in_Minutes='[1..159)', y='0')]

In [50]:
featureColumns = data.columns
featureColumns.remove('y')

In [51]:
# dataTrain,dataTest=data.randomSplit([0.7,0.3])
(mini_data, tmp) = data.randomSplit([0.01,0.99], 5)
(dataTrain, dataTest) = mini_data.randomSplit([0.8,0.2], 5)

Traceback (most recent call last):
  File "/home/huan/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/home/huan/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/home/huan/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/home/huan/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [52]:
print(dataTrain.count())
print(dataTest.count())

828


195


In [53]:
dataTrain.take(5)

[Row(Gender='Female', Customer_Type='Loyal Customer', Age='[15..23)', Type_of_Travel='Business travel', Class='Business', Flight_Distance='[1021...1516)', Inflight_wifi_service='3', Departure/Arrival_time_convenient='3', Gate_location='3', Food_and_drink='2', Online_boarding='2', Seat_comfort='2', Inflight_entertainment='2', On-board_service='3', Leg_room_service='2', Baggage_handling='5', Checkin_service='3', Inflight_service='5', Cleanliness='2', Arrival_Delay_in_Minutes='[..0)', y='1'),
 Row(Gender='Female', Customer_Type='Loyal Customer', Age='[15..23)', Type_of_Travel='Business travel', Class='Business', Flight_Distance='[1516...2011)', Inflight_wifi_service='3', Departure/Arrival_time_convenient='3', Gate_location='3', Food_and_drink='4', Online_boarding='4', Seat_comfort='4', Inflight_entertainment='4', On-board_service='4', Leg_room_service='3', Baggage_handling='5', Checkin_service='2', Inflight_service='2', Cleanliness='4', Arrival_Delay_in_Minutes='[1..159)', y='1'),
 Row(Ge

In [54]:
dataTrain

DataFrame[Gender: string, Customer_Type: string, Age: string, Type_of_Travel: string, Class: string, Flight_Distance: string, Inflight_wifi_service: string, Departure/Arrival_time_convenient: string, Gate_location: string, Food_and_drink: string, Online_boarding: string, Seat_comfort: string, Inflight_entertainment: string, On-board_service: string, Leg_room_service: string, Baggage_handling: string, Checkin_service: string, Inflight_service: string, Cleanliness: string, Arrival_Delay_in_Minutes: string, y: string]

In [55]:
dataTrain.show(vertical=True)

-RECORD 0--------------------------------------------
 Gender                            | Female          
 Customer_Type                     | Loyal Customer  
 Age                               | [15..23)        
 Type_of_Travel                    | Business travel 
 Class                             | Business        
 Flight_Distance                   | [1021...1516)   
 Inflight_wifi_service             | 3               
 Departure/Arrival_time_convenient | 3               
 Gate_location                     | 3               
 Food_and_drink                    | 2               
 Online_boarding                   | 2               
 Seat_comfort                      | 2               
 Inflight_entertainment            | 2               
 On-board_service                  | 3               
 Leg_room_service                  | 2               
 Baggage_handling                  | 5               
 Checkin_service                   | 3               
 Inflight_service           

In [56]:
# #chuyen sporadicValues sang panda_df
# sporadicValues = pd.DataFrame(sporadicValues.collect(), columns=dataTrain.columns)


In [57]:
# #Tao df theo chieu doc cac cot features thanh dang key-value
# def to_long(df,labelColumn):
#     cols,dtypes = zip(*((c,t) for (c,t) in df.dtypes if c not in labelColumn))
#     assert len(set(dtypes)) == 1,"All columns have to be of the same type"
#     kvs=explode(array([struct(lit).t(calias("key"), col(c).alias("val")) for c in cols])).alias("kvs")
#     return df.select(labelColumn+[kvs]).select(labelColumn+["kvs.key","kvs.val"])

In [58]:
# keyValueTrainDF = to_long(dataTrain,"y")
# keyValueTrainDF.registerTempTable("keyValueTrainDF")
# print("key-value train data")

In [59]:
def melt(
        df: DataFrame, 
        id_vars: Iterable[str], value_vars: Iterable[str], 
        var_name: str="variable", value_name: str="value") -> DataFrame:
    """Convert :class:`DataFrame` from wide to long format."""

    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = array(*(
        struct(lit(c).alias(var_name), col(c).alias(value_name)) 
        for c in value_vars))

    # Add to the DataFrame and explode
    _tmp = df.withColumn("_vars_and_vals", explode(_vars_and_vals))

    cols = id_vars + [
            col("_vars_and_vals")[x].alias(x) for x in [var_name, value_name]]
    return _tmp.select(*cols)

In [60]:
dataTrain_melted=melt(dataTrain, id_vars=['y'],var_name='independent_variables',value_vars=featureColumns)
dataTrain_melted.registerTempTable('TrainMelted')
dataTrain_melted.show()

+---+---------------------+---------------+
|  y|independent_variables|          value|
+---+---------------------+---------------+
|  1|               Gender|         Female|
|  1|        Customer_Type| Loyal Customer|
|  1|                  Age|       [15..23)|
|  1|       Type_of_Travel|Business travel|
|  1|                Class|       Business|
|  1|      Flight_Distance|  [1021...1516)|
|  1| Inflight_wifi_ser...|              3|
|  1| Departure/Arrival...|              3|
|  1|        Gate_location|              3|
|  1|       Food_and_drink|              2|
|  1|      Online_boarding|              2|
|  1|         Seat_comfort|              2|
|  1| Inflight_entertai...|              2|
|  1|     On-board_service|              3|
|  1|     Leg_room_service|              2|
|  1|     Baggage_handling|              5|
|  1|      Checkin_service|              3|
|  1|     Inflight_service|              5|
|  1|          Cleanliness|              2|
|  1| Arrival_Delay_in_...|     

In [61]:
train_frequencyCount = sqlContext.sql("SELECT independent_variables, value , y , COUNT(*) AS frequency FROM TrainMelted GROUP BY independent_variables, value , y")
train_frequencyCount.registerTempTable('TrainFrequency')
train_frequencyCount.show()

+---------------------+-----------------+---+---------+
|independent_variables|            value|  y|frequency|
+---------------------+-----------------+---+---------+
|     Leg_room_service|                0|  1|        2|
|                  Age|          [7..15)|  1|       10|
| Inflight_entertai...|                2|  1|       28|
|        Customer_Type|disloyal Customer|  1|       35|
|     Inflight_service|                5|  1|      136|
|      Checkin_service|                5|  0|       67|
|     Baggage_handling|                3|  1|       40|
|     On-board_service|                4|  1|      133|
| Inflight_wifi_ser...|                5|  1|       95|
|      Checkin_service|                1|  0|       83|
| Arrival_Delay_in_...|            [..0)|  0|      235|
|     Inflight_service|                1|  0|       38|
| Departure/Arrival...|                3|  1|       76|
|      Online_boarding|                4|  0|       90|
|                  Age|         [23..31)|  0|   

In [62]:
train_frequency_pd = pd.DataFrame(train_frequencyCount.collect(), columns = train_frequencyCount.columns)
_train_frequency_pd = sc.broadcast(train_frequency_pd)
_train_frequency_pd.value

,independent_variables,value,y,frequency
0,Leg_room_service,0,1,2
1,Age,[7..15),1,10
2,Inflight_entertainment,2,1,28
3,Customer_Type,disloyal Customer,1,35
4,Inflight_service,5,1,136
...,...,...,...,...
189,Gender,Female,0,223
190,Gate_location,3,0,144
191,Departure/Arrival_time_convenient,1,1,63
192,Food_and_drink,3,0,93


In [63]:
train_frequencyCount

DataFrame[independent_variables: string, value: string, y: string, frequency: bigint]

In [64]:
# classifcation=data.select('y').distinct().rdd.map(lambda x:x[0]).collect()
# classfication
classfications = dataTrain.select('y').distinct().rdd.map(lambda x: x[0]).collect()
_classfications = sc.broadcast(classfications)
_classfications.value

['0', '1']

In [65]:
_featureColumns=sc.broadcast(dataTrain.columns)
_featureColumns.value

['Gender',
 'Customer_Type',
 'Age',
 'Type_of_Travel',
 'Class',
 'Flight_Distance',
 'Inflight_wifi_service',
 'Departure/Arrival_time_convenient',
 'Gate_location',
 'Food_and_drink',
 'Online_boarding',
 'Seat_comfort',
 'Inflight_entertainment',
 'On-board_service',
 'Leg_room_service',
 'Baggage_handling',
 'Checkin_service',
 'Inflight_service',
 'Cleanliness',
 'Arrival_Delay_in_Minutes',
 'y']

In [66]:
#dem cac phan lop cua nhan va cac gia tri roi rac cua tung thuoc tinh
sporadicValues = dataTrain.agg(*(countDistinct(col(c)).alias(c) for c in dataTrain.columns))
sporadicValues = pd.DataFrame(sporadicValues.collect(), columns=dataTrain.columns).squeeze()
_sporadicValues = sc.broadcast(sporadicValues)
_sporadicValues.value
# sporadicValues
# dataTrain.agg(*(countDistinct(col(c)).alias(c) for c in dataTrain.columns))

Gender                               2
Customer_Type                        2
Age                                  9
Type_of_Travel                       2
Class                                3
Flight_Distance                      8
Inflight_wifi_service                6
Departure/Arrival_time_convenient    6
Gate_location                        5
Food_and_drink                       6
Online_boarding                      6
Seat_comfort                         5
Inflight_entertainment               5
On-board_service                     5
Leg_room_service                     6
Baggage_handling                     5
Checkin_service                      5
Inflight_service                     5
Cleanliness                          5
Arrival_Delay_in_Minutes             2
y                                    2
Name: 0, dtype: int64

In [67]:
n_df = sqlContext.sql("SELECT SUM(frequency) as n FROM TrainFrequency")
n = n_df.collect()[0]['n']
_n = sc.broadcast(n)
_n.value

16560

In [68]:
def classfication(row):
    p_results = []
    # so luong phan tu thuoc phan lop 0 va 1
    for classfication in _classfications.value:
        n_classfication= train_frequencyCount.select("frequency").where(train_frequencyCount.y==classfication).groupBy().sum().collect()[0][0];
        frequencies = []
        for i in range(0,len(row)-1):
            n_i = train_frequencyCount.select("frequency").where((train_frequencyCount.independent_variables==_featureColumns.value[i])&(train_frequencyCount.value==row[i])&(train_frequencyCount.y==classfication)).groupBy().sum().collect()[0][0]
            frequencies.append([n_i, _featureColumns.value[i]])
        p_classfication = float(n_classfication)/float(_n.value)
        for frequency in frequencies:
            if(frequency[0] !=0):
                p_classfication *=float(frequency[0])/float(n_classfication + _sporadicValues.value[frequency[1]])
            else:
                p_classfication *=float(frequency[0]+1)/float(n_classfication + _sporadicValues.value[frequency[1]])
        p_results.append(p_classfication)
    index = p_results.index(max(p_results))
    return _classfications.value[index]
# def classfication(row):
#     p_results = []
    
#     for classfication in _classfications.value:

#         n_classfication = _train_frequency_pd.value.loc[_train_frequency_pd.value['y'] == classfication, 'frequency'].sum()
#         frequencies = []
#         for i in range(0, len(row)-1):
#             n_i = train_frequency_pd.loc[
#                         (train_frequency_pd['independent_variables'] == _featureColumns.value[i]) &
#                         (train_frequency_pd['value'] == row[i]) & 
#                         (train_frequency_pd['y'] == classfication),
#                         'frequency'
#                     ].sum();

#             frequencies.append([n_i, _featureColumns.value[i]])
#         p_classfication = float(n_classfication) / float(_n.value)
#         for frequency in frequencies:
#             if(frequency[0] != 0):
#                 p_classfication *= float(frequency[0])/float(n_classfication + _sporadicValues.value[frequency[1]])
#             else:
#                 p_classfication *= float(frequency[0]+1)/float(n_classfication + _sporadicValues.value[frequency[1]])
#         p_results.append(p_classfication)
#     index = p_results.index(max(p_results))
#     return _classfications.value[index]

In [69]:
results = dataTest.rdd.map(classfication)
results

Traceback (most recent call last):
  File "/home/huan/spark-3.1.2-bin-hadoop3.2/python/pyspark/serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/home/huan/spark-3.1.2-bin-hadoop3.2/python/pyspark/cloudpickle/cloudpickle_fast.py", line 72, in dumps
    cp.dump(obj)
  File "/home/huan/spark-3.1.2-bin-hadoop3.2/python/pyspark/cloudpickle/cloudpickle_fast.py", line 540, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.RLock' object


PicklingError: Could not serialize object: TypeError: cannot pickle '_thread.RLock' object

In [70]:
def evaluate(test, results):
    n = len(results)
    n_correct = 0
    # chuyen sang pandas
    test_pd = pd.DataFrame(test.collect(), columns = test.columns)
    
    for i in range(0, n-1):
        if(results[i] == test_pd['y'][1]):
            n_correct += 1
    return float(n_correct)/float(n)

In [71]:
evaluate(dataTest, results.collect())

Traceback (most recent call last):
  File "/home/huan/spark-3.1.2-bin-hadoop3.2/python/pyspark/serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/home/huan/spark-3.1.2-bin-hadoop3.2/python/pyspark/cloudpickle/cloudpickle_fast.py", line 72, in dumps
    cp.dump(obj)
  File "/home/huan/spark-3.1.2-bin-hadoop3.2/python/pyspark/cloudpickle/cloudpickle_fast.py", line 540, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.RLock' object


PicklingError: Could not serialize object: TypeError: cannot pickle '_thread.RLock' object

In [ ]:
len(results.collect())

In [ ]:
dataTrain.groupBy("y").count().show()

In [ ]:
train_frequencyCount.groupBy("y").sum("frequency").show()


In [ ]:
train_frequencyCount.groupBy("independent_variables","y").sum("frequency").show()


In [ ]:
n_i =train_frequencyCount.select("frequency").where(train_frequencyCount.independent_variables==_featureColumns.value[0]).groupBy().sum().collect()[0][0]

In [ ]:
n_i =train_frequencyCount.select("frequency").where((train_frequencyCount.independent_variables==_featureColumns.value[0])&(train_frequencyCount.value==row[i])&(train_frequencyCount.y==classs)).groupBy().sum().collect()[0][0]

In [ ]:
n_i =train_frequencyCount.select("frequency").where((train_frequencyCount.independent_variables==_featureColumns.value[0])).show()